# Set Up

In [1]:
import bpy
import bmesh
from mathutils.bvhtree import BVHTree
from mathutils import Vector
import os
import pandas as pd
import numpy as np
from bpy_extras.object_utils import world_to_camera_view
import json

In [2]:
#----------------------------------------------------------------------
# BLENDER STAPLES
def render_and_log(render_file_path):
    bpy.context.view_layer.update()
    bpy.context.scene.render.filepath = render_file_path
    bpy.ops.render.render(write_still = True)

def config_GPU_and_render_quality():
    # ──────────── Make sure GPU is used ─────────────
    # 3.1 Set render engine
    bpy.context.scene.render.engine = 'CYCLES'

    # 3.2 Configure Cycles backend
    prefs = bpy.context.preferences.addons["cycles"].preferences
    prefs.compute_device_type = "CUDA"   # or "OPTIX", "ONEAPI", "HIP"
    prefs.get_devices()                  # populate prefs.devices

    # 3.3 Enable all non-CPU devices
    for dev in prefs.devices:
        if dev.type != 'CPU':
            dev.use = True

    # 3.4 Tell the scene to use GPU
    bpy.context.scene.cycles.device = 'GPU'

    # ──────────── 4) Render settings ─────────────
    # 8192 standard but we will use 512 for testing
    # bpy.context.scene.cycles.samples            = 8192
    # bpy.context.scene.render.resolution_percentage = 100
    bpy.context.scene.cycles.samples            = 1024
    bpy.context.scene.render.resolution_percentage = 50
    # bpy.context.scene.cycles.samples            = 256
    # bpy.context.scene.render.resolution_percentage = 25
    # bpy.context.scene.cycles.samples            = 32
    # bpy.context.scene.render.resolution_percentage = 5

    # ──────────── 5) Print scene & device info ─────────────
    print("Scene name:", bpy.context.scene.name)
    print("Engine      :", bpy.context.scene.render.engine)
    print("Backend     :", prefs.compute_device_type)
    print("Scene device:", bpy.context.scene.cycles.device)
    print("Available devices:")
    for d in prefs.devices:
        print(f"  - {d.name:25} type={d.type:4} use={d.use}")
    # ──────────── Rest of code ─────────────
#----------------------------------------------------------------------


In [3]:
#----------------------------------------------------------------------
# MASTER_DICT
# get all the important information you need
# that does not require opening the blender file
def getMasterDict(dataset_path, scene_id):
    # Conversion_dicts
    # dicts to convert between object-indicies to solve_state_json_names and blender_names
    indexToBlender = {}
    blenderToIndex = {}
    jsonToBlender = {}
    blenderToJson = {}

    # get all object indecies from the objectsegmentation array
    # frames/ObjectSegmentation/camera_0/ObjectSegmentation_0_0_0048_0.npy
    obj_seg_path = os.path.join( dataset_path, scene_id, "frames", "ObjectSegmentation", "camera_0", "ObjectSegmentation_0_0_0048_0.npy")
    obj_seg_arr = np.load(obj_seg_path)
    obj_seg_set = set( obj_seg_arr.flatten() )
    if 0 in obj_seg_set:
        obj_seg_set.remove(0) # remove 0 from set

    # go through objects.json, find all matching objects, and update the conversion_dicts
    obj_json_path = os.path.join( dataset_path, scene_id, "frames", "Objects", "camera_0", "Objects_0_0_0048_0.json")
    with open( obj_json_path, "r") as obj_json_file:
        obj_json_map = json.load(obj_json_file)
    
    for key, meta in obj_json_map.items():
        blender_obj_name = key
        obj_index = meta["object_index"]
        if obj_index in obj_seg_set:
            indexToBlender[obj_index] = blender_obj_name
            blenderToIndex[blender_obj_name] = obj_index
    
    # print("PRE-FILTERING: Blender-Index Conversions: ", "Number of enteries: ", len(indexToBlender))
    # print("IndexToBlender: ", indexToBlender)
    # print("BlenderToIndex: ", blenderToIndex)
    # print("\n", flush = True)

    # filter out unwanted objects
    filtered_list = [] # a list of (obj, index) to be removed
    filter_set = {"kitchen_0", "living-room_0", "dining-room_0", "bedroom_0", "bathroom_0"}
    for blender_obj_name, obj_index in blenderToIndex.items():
        for filter_item in filter_set:
            if filter_item in blender_obj_name:
                filtered_list.append( (blender_obj_name, obj_index) )
                break

    for blender_obj_name, obj_index in filtered_list:
        blenderToIndex.pop(blender_obj_name)
        indexToBlender.pop(obj_index)

    # populate dicts for blender_obj_names to and from solve_state_json_obj_names
    solve_state_json_path = os.path.join( dataset_path, scene_id, "coarse", "solve_state.json" )
    with open(solve_state_json_path, "r") as solve_state_json_file:
        solve_state_json_map = json.load(solve_state_json_file)
        solve_state_json_map = solve_state_json_map["objs"] # unpack objs
    for key, meta in solve_state_json_map.items():
        if meta['obj'] in blenderToIndex:
            json_obj_name = key
            blender_obj_name = meta['obj']
            blenderToJson[blender_obj_name] = json_obj_name
            jsonToBlender[json_obj_name] = blender_obj_name
    # json has less objects than index and blender
    for obj_blender_name in blenderToIndex.keys():
        if obj_blender_name not in blenderToJson:
            blenderToJson[obj_blender_name] = None

    # print("Filter: ", "removedCount:", len(filtered_list), "remainingCount", len(blenderToIndex))
    # print("removed: ", filtered_list)
    # print("remaining: ", blenderToIndex.keys())
    # print("POST-FILTER: (index: blender : json)")
    # for obj_index, blender_obj_name in indexToBlender.items():
    #     print(f"{obj_index}: {blender_obj_name} : {blenderToJson[blender_obj_name]}")
    # print("\n", flush = True)


    # for each object (blender_name), map it to a list of 'dependents' that depend on it
    # i.e. table should map to all the plates on the table
    # use blender names for objects
    dependents_map = {}

    for root in blenderToIndex.keys():
        # run bfs
        queue = [ root ]
        dependents = set() # fill it with all objects connected to parent
        visited = { root } # technically not needed since you can only depend on one thing and there are no cycles
        while queue:
            # process current object by adding it to dependents set
            curr_obj_blender = queue.pop(0)
            if curr_obj_blender != root: # don't include the root as a dependent
                dependents.add( curr_obj_blender )

            # put all eligible children in the queue

            # Objects.json condition...
            # accept children of curr_obj_blender
            for child_index in obj_json_map[curr_obj_blender]["children"]:
                if child_index in indexToBlender and indexToBlender[child_index] not in visited:
                    visited.add( indexToBlender[child_index] )
                    queue.append( indexToBlender[child_index] )
            
            # Solve_state.json condition...
            if not blenderToJson[curr_obj_blender]: # some blender objs are not listed in obj
                continue
            for key in jsonToBlender.keys():
                if jsonToBlender[key] in visited:
                    continue
                for rel in solve_state_json_map[key]["relations"]:
                    target_name = rel["target_name"]
                    relation_type = rel["relation"]["relation_type"]
                    child_tags = rel["relation"]["child_tags"]
                    if (target_name == blenderToJson[curr_obj_blender]
                            and relation_type == "StableAgainst"
                            and "Subpart(bottom)" in child_tags
                    ):
                        visited.add( jsonToBlender[key] )
                        queue.append( jsonToBlender[key] )
        dependents_map[ root ] = dependents

    # print out pretty formatted dependents_map
    # def set_default(obj):
    #     if isinstance(obj, set):
    #         return list(obj)
    #     raise TypeError
    # pretty_print = json.dumps( dependents_map, indent=2, default=set_default )
    # print( "Dependents_map: ", pretty_print )

    # categorize objects by groups
    # create maps from groups to objects and vice versa
    groupToObjects = {}
    objectToGroup = {}
    for blender_obj_name in blenderToIndex.keys():
        index = blender_obj_name.find("Factory")
        if index != -1:
            group_name = blender_obj_name[:index]
        else:
            group_name = blender_obj_name
        objectToGroup[blender_obj_name] = group_name
        if group_name in groupToObjects:
            groupToObjects[group_name].append(blender_obj_name)
        else:
            groupToObjects[group_name] = [ blender_obj_name ]
    
    masterDict = {
        # our 4 name conversion dictionaries
        "indexToBlender": indexToBlender,
        "blenderToIndex": blenderToIndex,
        "jsonToBlender": jsonToBlender,
        "blenderToJson": blenderToJson,

        # relational dependencies
        "dependents_map": dependents_map,
        "groupToObjects": groupToObjects,
        "objectToGroup": objectToGroup,

        "obj_seg_arr": obj_seg_arr, #from objectSegmentation.npy
    }
    return masterDict
#----------------------------------------------------------------------

# Initial Testing

In [3]:
output_folder = "/n/fs/obj-cv/experiment_project/experiments/objectSwapRelationshipExp/results" # for the data set of transformed images
dataset_path = "/n/fs/obj-cv/infinigen_project/savedDatasets/kitchenDataset94" # original dataset
index = 0

# read the pd as csv
df = pd.read_csv( "/n/fs/obj-cv/experiment_project/experiments/objectSwapRelationshipExp/KitchenSpaceToOvenFiltered.csv" )

# for each row, extract the source, smallObject, and dest
row = df.loc[index]
source, smallObject, dest = row["source"], row["smallObject"], row["dest"]
scene_id = row["sceneId"]

# open blender scene
blender_scene_path = os.path.join( dataset_path, scene_id, "fine", "scene.blend" )
bpy.ops.wm.open_mainfile(filepath = blender_scene_path)
print(flush=True)
config_GPU_and_render_quality()

Read blend: "/n/fs/obj-cv/infinigen_project/savedDatasets/kitchenDataset94/1420624e/fine/scene.blend"

Scene name: Scene
Engine      : CYCLES
Backend     : CUDA
Scene device: GPU
Available devices:
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - Intel Xeon Gold 5320 CPU @ 2.20GHz type=CPU  use=False
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
 

In [ ]:
img_name = "temp_img"
render_and_log( os.path.join( "/n/fs/obj-cv/experiment_project/experiments/objectSwapRelationshipExp", img_name ) )

Fra:1 Mem:313.61M (Peak 313.61M) | Time:00:00.93 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | CeilingLightFactory(4730346).spawn_asset(6552114)
Fra:1 Mem:313.62M (Peak 313.62M) | Time:00:00.93 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | WindowFactory(35558).spawn_asset(2)
Fra:1 Mem:313.62M (Peak 313.62M) | Time:00:00.93 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | WindowFactory(35558).spawn_asset(3)
Fra:1 Mem:313.62M (Peak 313.62M) | Time:00:00.93 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | WindowFactory(6349).spawn_asset(5)
Fra:1 Mem:313.65M (Peak 313.65M) | Time:00:00.93 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | WindowFactory(6349).spawn_asset(6)
Fra:1 Mem:313.66M (Peak 313.66M) | Time:00:00.93 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | WindowFactory(6349).spawn_asset(7)
Fra:1 Mem:313.67M (Peak 313.70M) | Time:00:00.93 | Mem:0.00M, Peak:0.00

In [4]:
def max_min_bound_box( obj, local = False ):
    for idx, corner in enumerate(obj.bound_box):
        if local:
            vec_corner = Vector(corner) # coordinates relative to object origin (obj.location)
        else:
            vec_corner = obj.matrix_world @ Vector(corner)
        if idx == 0:
            max_min_dict = {
                "x_max": vec_corner.x,
                "x_min": vec_corner.x,
                "y_max": vec_corner.y,
                "y_min": vec_corner.y,
                "z_max": vec_corner.z,
                "z_min": vec_corner.z
            }
        else:
            max_min_dict["x_max"] = max( max_min_dict["x_max"], vec_corner.x )
            max_min_dict["x_min"] = min( max_min_dict["x_min"], vec_corner.x )
            max_min_dict["y_max"] = max( max_min_dict["y_max"], vec_corner.y )
            max_min_dict["y_min"] = min( max_min_dict["y_min"], vec_corner.y )
            max_min_dict["z_max"] = max( max_min_dict["z_max"], vec_corner.z )
            max_min_dict["z_min"] = min( max_min_dict["z_min"], vec_corner.z )
    return max_min_dict

In [ ]:
# move the plate (smallObject) right above the oven (dest)
small_obj = bpy.data.objects[smallObject]
small_obj_mmD = max_min_bound_box(small_obj)
small_obj_mmD_local = max_min_bound_box(small_obj, True)

print(small_obj_mmD)
print(small_obj_mmD_local)
print(small_obj.location) # location is prob already world coorindates
# print(small_obj.matrix_world @ small_obj.location) # this is prob wrong

print("All coordinates of bound box (World Coordinates)")
for corner in small_obj.bound_box:
    print( small_obj.matrix_world @ Vector(corner) )
print("All coordinates of bound box (Local Coordinates)")
for corner in small_obj.bound_box:
    print( Vector(corner) )
x_axis_world = small_obj.matrix_world.to_3x3() @ Vector((1, 0, 0))
y_axis_world = small_obj.matrix_world.to_3x3() @ Vector((0, 1, 0))
z_axis_world = small_obj.matrix_world.to_3x3() @ Vector((0, 0, 1))
print(x_axis_world, y_axis_world, z_axis_world)


{'x_max': 7.772401332855225, 'x_min': 7.152478218078613, 'y_max': 5.932343482971191, 'y_min': 5.172405242919922, 'z_max': 1.4259662628173828, 'z_min': 1.0524342060089111}
{'x_max': 0.6199231743812561, 'x_min': 0.0, 'y_max': 0.7599380016326904, 'y_min': 0.0, 'z_max': 0.3735320568084717, 'z_min': -2.7755575615628914e-17}
<Vector (7.7724, 5.9323, 1.0524)>
All coordinates of bound box (World Coordinates)
<Vector (7.7724, 5.9323, 1.0524)>
<Vector (7.7724, 5.9323, 1.4260)>
<Vector (7.7724, 5.1724, 1.4260)>
<Vector (7.7724, 5.1724, 1.0524)>
<Vector (7.1525, 5.9323, 1.0524)>
<Vector (7.1525, 5.9323, 1.4260)>
<Vector (7.1525, 5.1724, 1.4260)>
<Vector (7.1525, 5.1724, 1.0524)>
All coordinates of bound box (Local Coordinates)
<Vector (0.0000, 0.0000, -0.0000)>
<Vector (0.0000, 0.0000, 0.3735)>
<Vector (0.0000, 0.7599, 0.3735)>
<Vector (0.0000, 0.7599, -0.0000)>
<Vector (0.6199, 0.0000, -0.0000)>
<Vector (0.6199, 0.0000, 0.3735)>
<Vector (0.6199, 0.7599, 0.3735)>
<Vector (0.6199, 0.7599, -0.0000)>

In [6]:
big_obj = bpy.data.objects[dest]
big_obj_mmD = max_min_bound_box( big_obj )
big_obj_mmD_local = max_min_bound_box( big_obj, True )

print(big_obj_mmD)
print(big_obj_mmD_local)
print(big_obj.location) # location is prob already world coorindates
print(big_obj.matrix_world @ big_obj.location) # this seems wrong


{'x_max': 5.724734306335449, 'x_min': 4.621462821960449, 'y_max': 6.4947099685668945, 'y_min': 5.17918586730957, 'z_max': 1.4444997310638428, 'z_min': 0.10759860277175903}
{'x_max': 1.258986234664917, 'x_min': -0.05653773248195648, 'y_max': 1.1032711267471313, 'y_min': 0.0, 'z_max': 1.3369011878967285, 'z_min': 0.0}
<Vector (5.7247, 5.2357, 0.1076)>
<Vector (0.4890, 10.9605, 0.2152)>


In [7]:
# align center of small_obj with big_obj for the x,y dimensions

# get center of small_obj x,y
small_obj_cx = (small_obj_mmD["x_min"] + small_obj_mmD["x_max"]) / 2
small_obj_cy = (small_obj_mmD["y_min"] + small_obj_mmD["y_max"]) / 2

# get center of big_obj x,y
big_obj_cx = (big_obj_mmD["x_min"] + big_obj_mmD["x_max"]) / 2
big_obj_cy = (big_obj_mmD["y_min"] + big_obj_mmD["y_max"]) / 2

# calculate displacement
disp_x = big_obj_cx - small_obj_cx
disp_y = big_obj_cy - small_obj_cy


# get bottom of small_obj to line up with top of big_obj
disp_z = big_obj_mmD["z_max"] - small_obj_mmD["z_min"]
print(disp_z)

# move small object origin
small_obj.location += Vector((disp_x, disp_y, disp_z))

0.39206552505493164


In [8]:
img_name = "temp_img"
render_and_log( os.path.join( "/n/fs/obj-cv/experiment_project/experiments/objectSwapRelationshipExp", img_name ) )

Fra:1 Mem:304.20M (Peak 304.20M) | Time:00:02.21 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | CeilingLightFactory(4730346).spawn_asset(6552114)
Fra:1 Mem:304.21M (Peak 304.21M) | Time:00:02.21 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | WindowFactory(35558).spawn_asset(2)
Fra:1 Mem:304.21M (Peak 304.21M) | Time:00:02.21 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | WindowFactory(35558).spawn_asset(3)
Fra:1 Mem:304.23M (Peak 304.23M) | Time:00:02.21 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | WindowFactory(35558).spawn_asset(4)
Fra:1 Mem:305.25M (Peak 306.26M) | Time:00:02.21 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | WindowFactory(6349).spawn_asset(5)
Fra:1 Mem:306.27M (Peak 306.27M) | Time:00:02.21 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | WindowFactory(6349).spawn_asset(7)
Fra:1 Mem:306.50M (Peak 306.50M) | Time:00:02.21 | Mem:0.00M, Peak:0.0

In [8]:
def add_rigid_body( obj, is_active ):
    # select object
    bpy.ops.object.select_all(action = "DESELECT")
    obj.select_set(True)
    bpy.context.view_layer.objects.active = obj
    
    bpy.ops.rigidbody.object_add()
    if is_active:
        obj.rigid_body.type = 'ACTIVE'
    else:
        obj.rigid_body.type = 'PASSIVE'

    # Optional: Set collision shape
    obj.rigid_body.collision_shape = 'CONVEX_HULL'

In [9]:
# https://blender.stackexchange.com/questions/71289/using-overlap-to-check-if-two-meshes-are-intersecting
def get_BVH_tree(obj):
    depsgraph = bpy.context.evaluated_depsgraph_get()
    eval_obj = obj.evaluated_get(depsgraph)
    mesh = eval_obj.to_mesh()

    bm = bmesh.new()
    bm.from_mesh(mesh)
    bm.transform(eval_obj.matrix_world)

    tree = BVHTree.FromBMesh(bm)

    bm.free()
    eval_obj.to_mesh_clear()
    return tree

# #get intersecting pairs
# inter = obj_now_BVHtree.overlap(obj_next_BVHtree)

# #if list is empty, no objects are touching
# if inter != []:
#     print(obj_now + " and " + obj_next + " are touching!")
# else:
#     print(obj_now + " and " + obj_next + " NOT touching!")

In [ ]:
bpy.context.scene.frame_set(1)

# give small obj a rigid body
# give large object a rigid body, set to passive
add_rigid_body(small_obj, is_active=True)
add_rigid_body(big_obj, is_active=False)

# go to frame 20
for frame in range(1, 16):
    bpy.context.scene.frame_set(frame)
    bpy.context.view_layer.update()  # Ensure updates are processed

    # render image
    # img_name = f"temp_img{frame}"
    # render_and_log( os.path.join( "/n/fs/obj-cv/experiment_project/experiments/objectSwapRelationshipExp", img_name ) )

    # add bmesh and get BVH tree for both objects
    small_tree = get_BVH_tree(small_obj)
    big_tree = get_BVH_tree(big_obj)
    inter = big_tree.overlap(small_tree)
    if inter:
        print(f"Collision at Frame {frame}")
        break

Fra:1 Mem:312.67M (Peak 312.67M) | Time:00:00.96 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | CeilingLightFactory(4730346).spawn_asset(6552114)
Fra:1 Mem:312.67M (Peak 312.67M) | Time:00:00.96 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | CeilingLightFactory(4730346).spawn_asset(20988)
Fra:1 Mem:312.67M (Peak 312.67M) | Time:00:00.96 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | CeilingLightFactory(4730346).spawn_asset(947399)
Fra:1 Mem:314.69M (Peak 314.70M) | Time:00:00.96 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | TapFactory(9203000).spawn_asset(807853)
Fra:1 Mem:314.71M (Peak 314.71M) | Time:00:00.96 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | LiteDoorFactory(5220690).spawn_asset(1)
Fra:1 Mem:314.72M (Peak 314.72M) | Time:00:00.97 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | PanelDoorFactory(1505269).spawn_asset(3)
Fra:1 Mem:314.78M (Peak 314.78

In [ ]:
# continuously lower small object until intersecting large object
for i in range(50):
    small_obj.location.z -= 0.01
    small_tree = get_BVH_tree(small_obj)
    big_tree = get_BVH_tree(big_obj)
    inter = big_tree.overlap(small_tree)
    if inter:
        print(f"Collision at at index {i}")
        break

In [59]:
img_name = "temp_img"
render_and_log( os.path.join( "/n/fs/obj-cv/experiment_project/experiments/objectSwapRelationshipExp", img_name ) )

Fra:4 Mem:410.67M (Peak 410.67M) | Time:00:00.92 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | CeilingLightFactory(4730346).spawn_asset(6552114)
Fra:4 Mem:410.67M (Peak 410.67M) | Time:00:00.92 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | PanelDoorFactory(1505269).spawn_asset(2)
Fra:4 Mem:410.67M (Peak 410.67M) | Time:00:00.92 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | GlassPanelDoorFactory(6323250).spawn_asset(0)
Fra:4 Mem:410.68M (Peak 410.69M) | Time:00:00.92 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | WindowFactory(35558).spawn_asset(0)
Fra:4 Mem:410.75M (Peak 410.77M) | Time:00:00.92 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | WindowFactory(35558).spawn_asset(2)
Fra:4 Mem:410.77M (Peak 410.77M) | Time:00:00.92 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | WindowFactory(35558).spawn_asset(3)
Fra:4 Mem:410.81M (Peak 410.81M) | Time:00:00.92 | Me

# Mini Solver Test

In [4]:
def max_min_bound_box( obj, local = False ):
    for idx, corner in enumerate(obj.bound_box):
        if local:
            vec_corner = Vector(corner) # coordinates relative to object origin (obj.location)
        else:
            vec_corner = obj.matrix_world @ Vector(corner)
        if idx == 0:
            max_min_dict = {
                "x_max": vec_corner.x,
                "x_min": vec_corner.x,
                "y_max": vec_corner.y,
                "y_min": vec_corner.y,
                "z_max": vec_corner.z,
                "z_min": vec_corner.z
            }
        else:
            max_min_dict["x_max"] = max( max_min_dict["x_max"], vec_corner.x )
            max_min_dict["x_min"] = min( max_min_dict["x_min"], vec_corner.x )
            max_min_dict["y_max"] = max( max_min_dict["y_max"], vec_corner.y )
            max_min_dict["y_min"] = min( max_min_dict["y_min"], vec_corner.y )
            max_min_dict["z_max"] = max( max_min_dict["z_max"], vec_corner.z )
            max_min_dict["z_min"] = min( max_min_dict["z_min"], vec_corner.z )
    return max_min_dict

In [5]:
def put_small_obj_over_big_obj( small_obj, big_obj ):
    """Uses bounding boxes to roughly move the small_obj on the big_obj"""
    small_obj_mmD = max_min_bound_box(small_obj)
    big_obj_mmD = max_min_bound_box( big_obj )
    
    # align center of small_obj with big_obj for the x,y dimensions
    # get center of small_obj x,y
    small_obj_cx = (small_obj_mmD["x_min"] + small_obj_mmD["x_max"]) / 2
    small_obj_cy = (small_obj_mmD["y_min"] + small_obj_mmD["y_max"]) / 2

    # get center of big_obj x,y
    big_obj_cx = (big_obj_mmD["x_min"] + big_obj_mmD["x_max"]) / 2
    big_obj_cy = (big_obj_mmD["y_min"] + big_obj_mmD["y_max"]) / 2

    # calculate displacement
    disp_x = big_obj_cx - small_obj_cx
    disp_y = big_obj_cy - small_obj_cy


    # get bottom of small_obj to line up with top of big_obj
    disp_z = big_obj_mmD["z_max"] - small_obj_mmD["z_min"]

    # # move small object origin
    # small_obj.location += Vector((disp_x, disp_y, disp_z))

    # copy & modify the world matrix
    mw = small_obj.matrix_world.copy()
    mw.translation += Vector((disp_x, disp_y, disp_z))

    # # Might need this if object "jumps"
    # small_obj.matrix_parent_inverse = small_obj.parent.matrix_world.inverted()

    # assign it back
    small_obj.matrix_world = mw

    bpy.context.view_layer.update()  # Ensure updates are processed

In [6]:
# Version 1. some helper methods are still used in V2

def solve_xy_on_flat_surface( small_obj, big_obj, dataset_path, scene_id, partitions=8 ):
    """Given small object's z coordinate is correct, move it along flat surface
    Returns 2D info matrixs where each element is tuple
    ( corners_in_clip, is_center_visible_by_raycast, count_ray_hits_to_camera, intersects_objects ) """

    # have the middle of the small_obj align with the top left corner of big_obj
    small_obj_mmD = max_min_bound_box(small_obj)
    big_obj_mmD = max_min_bound_box( big_obj )
    small_obj_cx = (small_obj_mmD["x_min"] + small_obj_mmD["x_max"]) / 2 # small object center
    small_obj_cy = (small_obj_mmD["y_min"] + small_obj_mmD["y_max"]) / 2
    big_obj_tlx, big_obj_tly = big_obj_mmD["x_min"], big_obj_mmD["y_max"] # top left corner large object
    origin_to_center = Vector(( small_obj_cx - small_obj.matrix_world.translation.x, 
                              small_obj_cy - small_obj.matrix_world.translation.y, 0 ))
    center_to_top_left_corner = Vector(( big_obj_tlx - small_obj_cx, big_obj_tly - small_obj_cy, 0 ))
    
    # Compute your “base” matrix once
    base_mw = small_obj.matrix_world.copy()
    base_mw.translation += origin_to_center + center_to_top_left_corner
    # Move object to the top–left only once
    small_obj.matrix_world = base_mw
    bpy.context.view_layer.update()

    width  = big_obj_mmD["x_max"] - big_obj_mmD["x_min"]
    height = big_obj_mmD["y_max"] - big_obj_mmD["y_min"]
    dx = width  / partitions
    dy = height / partitions

    # preallocate your score matrix
    info_matrix = [[None]*partitions for _ in range(partitions)]

    for i in range(partitions):
        for j in range(partitions):
            mw = base_mw.copy()
            mw.translation += Vector((i*dx, -j*dy, 0))
            small_obj.matrix_world = mw
            bpy.context.view_layer.update()

            # update info matrix:
            scene = bpy.context.scene
            cam = scene.camera
            info_matrix[j][i] = (corners_in_clip(scene, cam, small_obj), 
                                 is_center_visible_by_raycast(scene, cam, small_obj),
                                 count_ray_hits_to_camera(scene, cam, small_obj),
                                intersects_objects(small_obj, big_obj, dataset_path, scene_id) )
            
            # just for testing we render as well
            # img_name = f"temp_img(j={j}, i={i} )"
            # render_and_log( os.path.join( "/n/fs/obj-cv/experiment_project/experiments/objectSwapRelationshipExp", img_name ) )

    return info_matrix
    

def corners_in_clip(scene, cam_obj, obj):
    cam_data = cam_obj.data
    mat = obj.matrix_world
    
    # count how many corners are in the clip
    count = 0

    for corner in obj.bound_box:
        co_world = mat @ Vector(corner)
        ndc = world_to_camera_view(scene, cam_obj, co_world)
        # ndc.x/y in [0,1] is in‐frame; ndc.z is the depth on the view axis
        if (0.0 <= ndc.x <= 1.0 and
            0.0 <= ndc.y <= 1.0 and
            cam_data.clip_start < ndc.z < cam_data.clip_end):
            count += 1
    return count


def is_center_visible_by_raycast(scene, cam_obj, target_obj):
    """See if raycast from camera can hit center of object's bound box"""
    cam_loc = cam_obj.matrix_world.to_translation()
    # test against object’s bounding‐box center
    bbox_center = sum((Vector(c) for c in target_obj.bound_box), Vector()) / 8.0
    wc_center = target_obj.matrix_world @ bbox_center
    direction = (wc_center - cam_loc).normalized()

    result, hit_loc, hit_norm, _, hit_obj, _ = scene.ray_cast(
        depsgraph=bpy.context.evaluated_depsgraph_get(),
        origin=cam_loc,
        direction=direction
    )
    return result and hit_obj == target_obj

def count_ray_hits_to_camera(scene, cam_obj, target_obj, epsilon=1e-4):
    """
    Cast rays from the target_obj’s 8 bounding‑box corners
    toward the camera. Return how many of those 8 rays reach the 
    camera without hitting any meshes
    """
    depsgraph = bpy.context.evaluated_depsgraph_get()
    cam_loc   = cam_obj.matrix_world.to_translation()

    # build the 9 sample points in local space
    local_corners = [Vector(c) for c in target_obj.bound_box]

    hit_count = 0
    for lp in local_corners:
        # world‑space origin of the ray
        origin    = target_obj.matrix_world @ lp
        direction = (cam_loc - origin).normalized()

        # offset a bit so we don’t immediately self‑hit
        origin_offset = origin + direction * epsilon
        max_dist      = (cam_loc - origin_offset).length

        hit, hit_loc, hit_norm, face_idx, hit_obj, matrix = scene.ray_cast(
            depsgraph,
            origin=origin_offset,
            direction=direction,
            distance=max_dist
        )
        
        # If no mesh is hit then the ray hit the camera safely
        if not hit:
            hit_count += 1

    return hit_count

def intersects_objects(small_obj, big_obj, dataset_path, scene_id):
    """determine if small_obj intersects any dependents of the big_obj (except the small object itself)"""
    mDct = getMasterDict( dataset_path, scene_id )
    small_obj_name = small_obj.name
    big_obj_name = big_obj.name

    for dependent in mDct["dependents_map"][big_obj_name]:
        if dependent == small_obj_name: # don't want to check if intersects with self
            continue
        if aabb_intersect( small_obj, bpy.data.objects[dependent] ):
            return True
    return False

def aabb_intersect(obj_a, obj_b, check_z=True):
    mmA = max_min_bound_box(obj_a)
    mmB = max_min_bound_box(obj_b)
    overlap_x = mmA["x_max"] >= mmB["x_min"] and mmB["x_max"] >= mmA["x_min"]
    overlap_y = mmA["y_max"] >= mmB["y_min"] and mmB["y_max"] >= mmA["y_min"]
    if not (overlap_x and overlap_y):
        return False
    if check_z:
        overlap_z = mmA["z_max"] >= mmB["z_min"] and mmB["z_max"] >= mmA["z_min"]
        return overlap_z
    return True


In [7]:
# version 2
# ***over writes "solve_xy_on_flat_surface" from above
# "solve_xy_on_flat_surface" still uses the helper functions from above though

def raycast_top_grid( obj, x_min, x_max, y_min, y_max, z_origin, partitions=100 ):
    """ castsrays from a grid defined by x_min, x_max, y_min, y_max, z_origin, partitions
     reutrns an array of all vertices of obj that were hit """
    hits = []
    depsgraph = bpy.context.evaluated_depsgraph_get()
    scene = bpy.context.scene
    direction = Vector((0,0,-1))
    
    for i in range(partitions):
        for j in range(partitions):
            x_origin = x_min + (x_max - x_min) * (i/(partitions-1))
            y_origin = y_min + (y_max - y_min) * (j/(partitions-1))
            origin = Vector( (x_origin, y_origin, z_origin) )

            result, hit_loc, hit_norm, hit_idx, hit_obj, hit_mat = scene.ray_cast(
                depsgraph, origin + direction*1e-3, direction
            )
            if result:
                hits.append( Vector((x_origin, y_origin, hit_loc.z)) )
    return hits


def find_top_plane( hits ):
    """ takes the hits array from raycast_top_grid and returns a flat plane using
    the median z-coordinates, and min and max x,y coordinates that are within eps
    away from the median z-coordinate"""

    # if somehow all the rays missed and hits is empty
    if not hits:
        raise ValueError("No ray hits—can't compute top plane.")

    hits_x = sorted(hits, key=lambda vec: vec.x)
    hits_y = sorted(hits, key=lambda vec: vec.y)
    hits_z = sorted(hits, key=lambda vec: vec.z)
    z_median = hits_z[ len(hits)//2 ].z

    # filter out points that are more than epsilon away from z-median
    eps = 1e-3
    hits_x = [ point for point in hits_x if abs(point.z - z_median) < eps ]
    hits_y = [ point for point in hits_y if abs(point.z - z_median) < eps ]

    x_min = hits_x[0].x
    x_max = hits_x[len(hits_x)-1].x
    y_min = hits_y[0].y
    y_max = hits_y[len(hits_y)-1].y

    return ( x_min, x_max, y_min, y_max, z_median )


def solve_xy_on_flat_surface( small_obj, big_obj, top_plane, dataset_path, scene_id, partitions=10 ):
    """Given a top_plane, align small_obj's bottom z to top_plane's z. Then slide small_obj's xy coordinates
    along the plane.
    Returns 2D info matrixs where each element is tuple
    ( corners_in_clip, is_center_visible_by_raycast, count_ray_hits_to_camera, intersects_objects ) 
    
    [0, partitions] is the interval used to slide objects around in the x and y axis
    
    top_plane should be in the form ( x_min, x_max, y_min, y_max, z_median )
    which is what find_top_plane returns
    """

    tp_x_min, tp_x_max, tp_y_min, tp_y_max, tp_z_median = top_plane

    # have the middle of the small_obj align with the top left corner of top_plane
    small_obj_mmD = max_min_bound_box(small_obj)
    small_obj_cx = (small_obj_mmD["x_min"] + small_obj_mmD["x_max"]) / 2 # small object center
    small_obj_cy = (small_obj_mmD["y_min"] + small_obj_mmD["y_max"]) / 2
    center_to_top_left_corner = Vector(( tp_x_min - small_obj_cx, tp_y_max - small_obj_cy, 0 ))

    # also need a transform to align bottom of small_obj to the top_plane
    vec_z = Vector( (0,0, tp_z_median - small_obj_mmD["z_min"] ) )
    
    # Compute your “base” matrix once
    base_mw = small_obj.matrix_world.copy()
    base_mw.translation += center_to_top_left_corner + vec_z
    # Move object to the top–left only once
    small_obj.matrix_world = base_mw
    bpy.context.view_layer.update()

    width  = tp_x_max - tp_x_min
    height = tp_y_max - tp_y_min
    dx = width  / partitions
    dy = height / partitions

    # preallocate your score matrix
    info_matrix = [[None]*(partitions+1) for _ in range(partitions+1)]

    for i in range(partitions + 1):
        for j in range(partitions + 1):
            mw = base_mw.copy()
            mw.translation += Vector((i*dx, -j*dy, 0))
            small_obj.matrix_world = mw
            bpy.context.view_layer.update()

            # update info matrix:
            scene = bpy.context.scene
            cam = scene.camera
            info_matrix[j][i] = (corners_in_clip(scene, cam, small_obj), 
                                 is_center_visible_by_raycast(scene, cam, small_obj),
                                 count_ray_hits_to_camera(scene, cam, small_obj),
                                intersects_objects(small_obj, big_obj, dataset_path, scene_id),
                                 mw )
            
            # just for testing we render as well
            # img_name = f"temp_img(j={j}, i={i} )"
            # render_and_log( os.path.join( "/n/fs/obj-cv/experiment_project/experiments/objectSwapRelationshipExp", img_name ) )

    return info_matrix


def align_small_object_with_parenting( parent, child ):
    """ Used to have child's axis line up with parent's axis
     Front is +X in infinigen """
    child.parent = parent

    # zero out child transforms
    child.matrix_basis.identity()
    
    # — Default: “no inverse” → child inherits parent’s transform immediately
    child.matrix_parent_inverse.identity()

    bpy.context.view_layer.update()

In [8]:
# specific add on for kitchenSpace
def largest_z_gap(obj):
    # 1) Collect all the Z‐coordinates
    zs = []
    
    # transform every 50 local vertex into world space:
    for i in range( 0, len(obj.data.vertices), 50 ):
        v = obj.data.vertices[i]
        co_world = obj.matrix_world @ v.co
        zs.append(co_world.z)

    # 2) Sort them
    zs_sorted = sorted(zs)

    # 3) Walk the sorted list and track the maximum gap
    max_gap = 0.0
    max_pair = (None, None)
    for a, b in zip(zs_sorted, zs_sorted[1:]):
        gap = b - a
        if gap > max_gap:
            max_gap = gap
            max_pair = (a, b)

    return max_gap, max_pair

# use something like (max_pair[0] + max_pair[1])/2 to float above the bottom counter

In [56]:
# add on to place object in best location

# info_matrix[j][i] = (corners_in_clip(scene, cam, small_obj), 
#                      is_center_visible_by_raycast(scene, cam, small_obj),
#                      count_ray_hits_to_camera(scene, cam, small_obj),
#                     intersects_objects(small_obj, big_obj, dataset_path, scene_id),
#                      mw )

def put_object_in_best_loc( obj, info_matrix ):
    """ Given an info matrix for an obj, move the obj to the 'best' location """
    score_list = [] # element = ( i, j, score )

    n = len( info_matrix )
    for i in range( n ):
        for j in range( n ):
            score = 0
            info = info_matrix[j][i] # info matrix is index x,y or j,i

            # if nothing is in clip, instant -1000
            if info[0] == 0:
                score_list.append( (i,j, -1000) )
                continue
            else:
                # get 200 pts for every vertex in clip
                score += 200 * info[0]
            
            # if center is visible +500
            if info[1]:
                score += 500
            
            # if intersecting other objects -750
            if info[3]:
                score -= 750
            
            # extra 100 for each corner visible
            score += info[2] * 100

            # edge case: center and all corners not visible, instant -1000
            if not info[1] and info[2]==0:
                score_list.append( (i,j, -1000) )
                continue

            # penalty for being on the outer edge -100
            if i==0 or i==n-1 or j==0 or j==n-1:
                score -= 100
            
            # additional scaling penalty for being away from center up to -500
            units_from_center = abs(i + j - n)
            score -= (units_from_center / n) * 500

            # add to score list
            score_list.append( (i, j, score) )
    
    # get the tuple corresponding to the max score and move object there
    best_tuple = max(score_list, key=lambda x: x[2])
    best_i, best_j, best_score = best_tuple
    mw = info_matrix[best_j][best_i][4]
    obj.matrix_world = mw
    bpy.context.view_layer.update()

    # returns scorelist and best_tuple for analysis purposes
    return score_list, best_tuple



v1: hope that putting the object on top lines everything up

In [20]:
dataset_path = "/n/fs/obj-cv/infinigen_project/savedDatasets/diningRoom99"
scene_id = "1cc42ce1"

# open blender scene
blender_scene_path = os.path.join( dataset_path, scene_id, "fine", "scene.blend" )
bpy.ops.wm.open_mainfile(filepath = blender_scene_path)
print(flush=True)
config_GPU_and_render_quality()

# GOAL: Move wine glass to table
wineglass_name = "WineglassFactory(1028783).spawn_asset(2729829)"
table_name = "TableDiningFactory(5721013).spawn_asset(481368)"
wineglass = bpy.data.objects[wineglass_name]
table = bpy.data.objects[table_name]

Read blend: "/n/fs/obj-cv/infinigen_project/savedDatasets/diningRoom99/1cc42ce1/fine/scene.blend"

Scene name: Scene
Engine      : CYCLES
Backend     : CUDA
Scene device: GPU
Available devices:
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - Intel Xeon Gold 5320 CPU @ 2.20GHz type=CPU  use=False
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - N

In [66]:
# Render to temp
img_name = "temp_img"
render_and_log( os.path.join( "/n/fs/obj-cv/experiment_project/experiments/objectSwapRelationshipExp", img_name ) )

Fra:1 Mem:252.11M (Peak 252.11M) | Time:00:00.87 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | CeilingLightFactory(8723727).spawn_asset(2390609)
Fra:1 Mem:252.12M (Peak 252.12M) | Time:00:00.87 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | LiteDoorFactory(178947).spawn_asset(1)
Fra:1 Mem:252.12M (Peak 252.12M) | Time:00:00.87 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | WindowFactory(4459).spawn_asset(0)
Fra:1 Mem:252.13M (Peak 252.13M) | Time:00:00.88 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | WindowFactory(4459).spawn_asset(2)
Fra:1 Mem:252.16M (Peak 252.16M) | Time:00:00.88 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | WindowFactory(4459).spawn_asset(4)
Fra:1 Mem:252.20M (Peak 252.20M) | Time:00:00.88 | Mem:0.00M, Peak:0.00M | Scene, ViewLayer | Synchronizing object | WindowFactory(4459).spawn_asset(5)
Fra:1 Mem:252.21M (Peak 252.21M) | Time:00:00.88 | Mem:0.00M, Peak:0.

In [8]:
# move wineglass on table
put_small_obj_over_big_obj( wineglass, table )

In [10]:
info_matrix = solve_xy_on_flat_surface( wineglass, table, dataset_path, scene_id )

In [13]:
info_matrix[0][0]
info_matrix[5][5]

(8, True, 6, True)

V2 Way of Finding Top Plane

In [34]:
dataset_path = "/n/fs/obj-cv/infinigen_project/savedDatasets/kitchenDataset94"
scene_id = "1420624e"

# open blender scene
blender_scene_path = os.path.join( dataset_path, scene_id, "fine", "scene.blend" )
bpy.ops.wm.open_mainfile(filepath = blender_scene_path)
print(flush=True)
config_GPU_and_render_quality()

# GOAL: Move microwave glass to oven
oven_name = "OvenFactory(2033892).spawn_asset(8574077)"
microwave_name = "MicrowaveFactory(1849906).spawn_asset(842634)"
oven = bpy.data.objects[oven_name]
microwave = bpy.data.objects[microwave_name]

Read blend: "/n/fs/obj-cv/infinigen_project/savedDatasets/kitchenDataset94/1420624e/fine/scene.blend"

Scene name: Scene
Engine      : CYCLES
Backend     : CUDA
Scene device: GPU
Available devices:
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - Intel Xeon Gold 5320 CPU @ 2.20GHz type=CPU  use=False
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
 

In [23]:
put_small_obj_over_big_obj( microwave, oven )

In [27]:
# degugging
def make_plane( corners ):
    # 1) Your four corner coordinates (order them consistently—here: bottom‑left, bottom‑right, top‑right, top‑left)
#    corners = [
#        Vector((-1.0, -1.0, 0.0)),  # v0
#        Vector(( 1.0, -1.0, 0.0)),  # v1
#        Vector(( 1.0,  1.0, 0.0)),  # v2
#        Vector((-1.0,  1.0, 0.0)),  # v3
#    ]

    # 2) One face that uses those verts in order
    faces = [(0, 1, 2, 3)]

    # 3) Create a new mesh datablock and fill it
    mesh = bpy.data.meshes.new(name="MyPlaneMesh")
    mesh.from_pydata(corners, [], faces)
    mesh.update()

    # 4) Create an object that uses this mesh
    obj = bpy.data.objects.new("MyPlane", mesh)

    # 5) Link the object into the current collection / scene
    col = bpy.context.collection
    col.objects.link(obj)

    # 6) (Optional) make it active and select it
    bpy.context.view_layer.objects.active = obj
    obj.select_set(True)

oven_mmD = max_min_bound_box( oven )
hits = raycast_top_grid( oven, oven_mmD["x_min"], oven_mmD["x_max"], oven_mmD["y_min"], oven_mmD["y_max"], oven_mmD["z_max"] )
top_plane = find_top_plane( hits )
x_min, x_max, y_min, y_max, z_median = top_plane
corners = [
    Vector((x_min, y_min, z_median)),
    Vector((x_max, y_min, z_median)),
    Vector((x_max, y_max, z_median)),
    Vector((x_min, y_max, z_median))
]
make_plane(corners)

In [38]:
# orient the mircowave
align_small_object_with_parenting( oven, microwave )

# get oven bound_box
oven_mmD = max_min_bound_box( oven )

# raycast to get hit vertices
hits = raycast_top_grid( oven, oven_mmD["x_min"], oven_mmD["x_max"], oven_mmD["y_min"], oven_mmD["y_max"], oven_mmD["z_max"] )

# convert collection of vectices to a plane (5-tuple)
top_plane = find_top_plane( hits )

# get info maxtrix using solver
info_matrix = solve_xy_on_flat_surface( microwave, oven, top_plane, dataset_path, scene_id )


ReferenceError: StructRNA of type Object has been removed

In [32]:
info_matrix

[[(3, True, 7, False),
  (4, True, 7, False),
  (4, True, 6, False),
  (5, True, 7, False),
  (6, True, 6, False),
  (6, True, 6, False),
  (8, True, 6, False),
  (8, True, 6, False),
  (8, True, 6, False),
  (8, True, 6, False),
  (8, True, 7, False)],
 [(4, True, 7, False),
  (4, True, 7, False),
  (4, True, 6, False),
  (6, True, 6, False),
  (6, True, 6, False),
  (6, True, 6, False),
  (8, True, 6, False),
  (8, True, 6, False),
  (8, True, 6, False),
  (8, True, 6, False),
  (8, True, 7, False)],
 [(4, True, 7, False),
  (4, True, 7, False),
  (4, True, 7, False),
  (6, True, 7, False),
  (6, True, 6, False),
  (7, True, 6, False),
  (8, True, 6, False),
  (8, True, 6, False),
  (8, True, 6, False),
  (8, True, 6, False),
  (8, True, 7, False)],
 [(4, True, 7, False),
  (4, True, 7, False),
  (6, True, 7, False),
  (6, True, 7, False),
  (6, True, 6, False),
  (8, True, 6, False),
  (8, True, 6, False),
  (8, True, 6, False),
  (8, True, 6, False),
  (8, True, 6, False),
  (8, Tr

specfic modification for kitchenSpace (finding the lower counter top)

In [10]:
dataset_path = "/n/fs/obj-cv/infinigen_project/savedDatasets/kitchenDataset94"
scene_id = "2dbefffc"

# open blender scene
blender_scene_path = os.path.join( dataset_path, scene_id, "fine", "scene.blend" )
bpy.ops.wm.open_mainfile(filepath = blender_scene_path)
print(flush=True)
config_GPU_and_render_quality()

# GOAL: Move wine glass to table
plant_name = "LargePlantContainerFactory(949299).spawn_asset(2608831)"
kitchenSpace_name = "KitchenSpaceFactory(3989955).spawn_asset(8637063)"
plant = bpy.data.objects[plant_name]
kitchenSpace = bpy.data.objects[kitchenSpace_name]

Read blend: "/n/fs/obj-cv/infinigen_project/savedDatasets/kitchenDataset94/2dbefffc/fine/scene.blend"

Scene name: Scene
Engine      : CYCLES
Backend     : CUDA
Scene device: GPU
Available devices:
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - Intel Xeon Gold 5320 CPU @ 2.20GHz type=CPU  use=False
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
 

In [16]:
# orient the plant
align_small_object_with_parenting( kitchenSpace, plant )

# get kitchenSpace bound_box
kitchenSpace_mmD = max_min_bound_box( kitchenSpace )

# SPECIAL: Update kitchenSpace upper bound box to only include bottom counter
max_gap, max_pair = largest_z_gap( kitchenSpace )
kitchenSpace_mmD["z_max"] = ( max_pair[0] + max_pair[1] ) / 2

# raycast to get hit vertices
hits = raycast_top_grid( kitchenSpace, kitchenSpace_mmD["x_min"], kitchenSpace_mmD["x_max"], 
                        kitchenSpace_mmD["y_min"], kitchenSpace_mmD["y_max"], kitchenSpace_mmD["z_max"] )

# convert collection of vectices to a plane (5-tuple)
top_plane = find_top_plane( hits )

# get info maxtrix using solver
info_matrix = solve_xy_on_flat_surface( plant, kitchenSpace, top_plane, dataset_path, scene_id )


NameError: name 'kitchenSpace' is not defined

Testing Best Object Alignment

In [63]:
# dataset_path = "/n/fs/obj-cv/infinigen_project/savedDatasets/kitchenDataset94"
# scene_id = "2dbefffc"

# # open blender scene
# blender_scene_path = os.path.join( dataset_path, scene_id, "fine", "scene.blend" )
# bpy.ops.wm.open_mainfile(filepath = blender_scene_path)
# print(flush=True)
# config_GPU_and_render_quality()

# # GOAL: Move wine glass to table
# small_obj_name = "LargePlantContainerFactory(949299).spawn_asset(2608831)"
# big_obj_name = "KitchenSpaceFactory(3989955).spawn_asset(8637063)"
# small_obj = bpy.data.objects[small_obj_name]
# big_obj = bpy.data.objects[big_obj_name]

# dataset_path = "/n/fs/obj-cv/infinigen_project/savedDatasets/kitchenDataset94"
# scene_id = "1420624e"


# # open blender scene
# blender_scene_path = os.path.join( dataset_path, scene_id, "fine", "scene.blend" )
# bpy.ops.wm.open_mainfile(filepath = blender_scene_path)
# print(flush=True)
# config_GPU_and_render_quality()

# # GOAL: Move microwave glass to oven
# big_obj_name = "OvenFactory(2033892).spawn_asset(8574077)"
# small_obj_name = "MicrowaveFactory(1849906).spawn_asset(842634)"
# big_obj = bpy.data.objects[big_obj_name]
# small_obj = bpy.data.objects[small_obj_name]


dataset_path = "/n/fs/obj-cv/infinigen_project/savedDatasets/kitchenDataset94"
scene_id = "2c653f74"

# open blender scene
blender_scene_path = os.path.join( dataset_path, scene_id, "fine", "scene.blend" )
bpy.ops.wm.open_mainfile(filepath = blender_scene_path)
print(flush=True)
config_GPU_and_render_quality()

# GOAL: Move microwave glass to oven
big_obj_name = "LargeShelfFactory(2741555).spawn_asset(1952064)"
small_obj_name = "BottleFactory(1493237).spawn_asset(8652564)"
big_obj = bpy.data.objects[big_obj_name]
small_obj = bpy.data.objects[small_obj_name]


Read blend: "/n/fs/obj-cv/infinigen_project/savedDatasets/kitchenDataset94/2c653f74/fine/scene.blend"

Scene name: Scene
Engine      : CYCLES
Backend     : CUDA
Scene device: GPU
Available devices:
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - Intel Xeon Gold 5320 CPU @ 2.20GHz type=CPU  use=False
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
 

In [64]:
# orient the small_obj
align_small_object_with_parenting( big_obj, small_obj )

# get kitchenSpace bound_box
big_mmD = max_min_bound_box( big_obj )

# SPECIAL: Update kitchenSpace upper bound box to only include bottom counter
if "KitchenSpace" in big_obj.name:
    max_gap, max_pair = largest_z_gap( big_obj )
    big_mmD["z_max"] = ( max_pair[0] + max_pair[1] ) / 2

# raycast to get hit vertices
hits = raycast_top_grid( big_obj, big_mmD["x_min"], big_mmD["x_max"], 
                        big_mmD["y_min"], big_mmD["y_max"], big_mmD["z_max"] )

# convert collection of vectices to a plane (5-tuple)
top_plane = find_top_plane( hits )

# get info maxtrix using solver
info_matrix = solve_xy_on_flat_surface( small_obj, big_obj, top_plane, dataset_path, scene_id )

# using info matrix place small object in the best location
score_list, best_tuple = put_object_in_best_loc( small_obj, info_matrix )

In [65]:
score_list
best_tuple
# score_list[5*11 + 5]
# info_matrix[5][5]
# info_matrix[9][2]

(1, 9, 2654.5454545454545)

# Export Room

In [7]:
dataset_path = "/n/fs/obj-cv/infinigen_project/savedDatasets/kitchenDataset94"
# scene_id = "1e122544"
# scene_id = "1420624e"
scene_id = "55ea34ab" # debug this

# SlideShow Image
dataset_path = "/n/fs/obj-cv/infinigen_project/Backups/CopyBackups/BatchTest1/my_dataset"
scene_id = "112d1667"

# dining room image
dataset_path = "/n/fs/obj-cv/infinigen_project/savedDatasets/diningRoom99"
scene_id = "7f9ecccd"

# bathroom image
dataset_path = "/n/fs/obj-cv/infinigen_project/savedDatasets/bathroom97"
scene_id = "169a7ea3"

# bedroom image
dataset_path = "/n/fs/obj-cv/infinigen_project/savedDatasets/bedroom96"
scene_id = "17349b62"
scene_id = "12fd0dc7" # simple book case

# livingroom image
dataset_path = "/n/fs/obj-cv/infinigen_project/savedDatasets/livingRoom99"
scene_id = "103ade0b"
scene_id = "167b03df"
scene_id = "1801d554"

# open blender scene
blender_scene_path = os.path.join( dataset_path, scene_id, "fine", "scene.blend" )
bpy.ops.wm.open_mainfile(filepath = blender_scene_path)
print(flush=True)
config_GPU_and_render_quality()

# open object.json
obj_json_path = os.path.join( dataset_path, scene_id, "frames", "Objects", "camera_0", "Objects_0_0_0048_0.json")
with open( obj_json_path, "r") as obj_json_file:
    obj_json_map = json.load(obj_json_file)

# GOAL: Delete everything that is not present in objects json
all_objs = list( bpy.data.objects )
for obj in all_objs:
    if obj.name not in obj_json_map.keys():
        bpy.data.objects.remove(obj, do_unlink=True)
# if you want to clear all orphaned data (might want to keep parents though)
# bpy.ops.outliner.orphans_purge(do_local_ids=True, do_linked_ids=True, do_recursive=True)

bpy.ops.wm.save_as_mainfile(
    filepath="/n/fs/obj-cv/experiment_project/experiments/objectSwapRelationshipExp/new_scene.blend",
    check_existing=True,    # prompt or overwrite if it already exists
    compress=True           # write with built‑in compression
)


Read blend: "/n/fs/obj-cv/infinigen_project/savedDatasets/livingRoom99/1801d554/fine/scene.blend"

Scene name: Scene
Engine      : CYCLES
Backend     : CUDA
Scene device: GPU
Available devices:
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - NVIDIA L40                type=CUDA use=True
  - Intel Xeon Gold 5320 CPU @ 2.20GHz type=CPU  use=False
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - NVIDIA L40                type=OPTIX use=True
  - N

{'FINISHED'}